In [32]:
from __future__ import division
from __future__ import print_function

import os
import random
import numpy as np
import cv2
import tensorflow
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=1
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
from natsort import natsorted
import os
import numpy as np
path="/home/tali/Desktop/PURVA_CHINIYA/IAMhand2printv5"
parameters = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"
global params
params = {'path' : "/home/tali/Desktop/PURVA_CHINIYA/IAMhand2printv5",
          'model_path': '/home/tali/Desktop/PURVA_CHINIYA',
          'Batch_Size': 50,
          'imgSize':(128,32),
          'num_classes':len(parameters)+1,
          'lr': 0.0001,
          'num_epochs':1000,
          'width_thresh':250}
def get_file_paths(path=params['path']):
    paths = np.array(natsorted([os.path.join(root, file) for root, dirs, files in os.walk(path) for file in files]))
    WordList = np.array([os.path.basename(i).split('_')[-1][:-4] for i in paths])
    index = np.random.permutation(len(paths))
    paths = paths[index]
    WordList = WordList[index]
    return paths[:int(len(paths) * .7)], paths[int(len(paths) * .7):], WordList[:int(len(WordList) * .7)], WordList[int(
        len(WordList) * .7):]
def load_img(path):
    img_ = cv2.imread(path)
    [r, c] = img_.shape[:2]
    r_ = 32
    c_ = round(c/r * r_)
    if c_ > 250:
        c_ = 250
    img_ = cv2.resize(img_, (c_,r_))  
    
    img_ = cv2.cvtColor(img_, cv2.COLOR_BGR2GRAY)
    img_ = cv2.copyMakeBorder(img_, 0, 0, 0, 250-c_, cv2.BORDER_CONSTANT, value=(255,255,255))
    return img_/255,c_
def Load_Batch(batch_paths):
    batch_data = []
    width=[]
    for path in batch_paths:
        img_,seq = load_img(path)
        batch_data.append(img_)
        width.append(seq)
    return np.array(batch_data),np.array(seq)
X_train,X_test,Y_train,Y_test=get_file_paths(path)

print("length of training images",len(X_train))
print("length of testing images",len(X_test))
print("length of training labels",len(Y_train))
print("length of testing labels",len(Y_test))



import cv2
Test_array,Test_width=Load_Batch(X_test)
Train_array,Train_width=Load_Batch(X_train)
print(Train_array.shape)
print(Test_array.shape)



env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=1
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6652478552719295967
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 3496374497123985203
physical_device_desc: "device: XLA_CPU device"
]
length of training images 57579
length of testing images 24678
length of training labels 57579
length of testing labels 24678
(57579, 32, 250)
(24678, 32, 250)


In [33]:

def ResNetCNN():
    
    with tf.variable_scope("Network"):
        with tf.variable_scope("CNN"):
            def block(x,n_output, upscale=True):
                h = tf.layers.batch_normalization(x)
                h = tf.nn.relu(h)
                h=tf.layers.conv2d(inputs=h,filters=n_output,strides=1,kernel_size=[3,3],padding="same",activation=tf.nn.relu)
                # second pre-activation
                h = tf.layers.batch_normalization(h)
                h = tf.nn.relu(h)
                h = tf.layers.conv2d(h, filters=n_output, strides=1,kernel_size=[3,3], padding='same')
                if upscale:
                    # 1x1 conv2d
                    f =tf.layers.conv2d(x,kernel_size=1, filters=n_output, strides=1, padding='same')
                else:
                    # identity
                    f = x
                # F_l(x) = f(x) + H_l(x):
                t=tf.math.add(f, h)
                return t
            Input = tf.placeholder(dtype=tf.float32, shape=[None, 32, params['width_thresh'], 3], name='Input')
            Seq_Len = tf.placeholder(dtype=tf.int32, shape=[None], name='Seq_len')
            # input_layer = tf.placeholder(dtype=tf.float32, shape=[None, 32, params['width_thresh'], 3], name='Input')
            conv1=tf.layers.conv2d(Input,filters=64,kernel_size=[3,3],padding="same",activation=tf.nn.relu)

            conv2=tf.layers.conv2d(conv1,filters=128,kernel_size=[3,3],padding="same",activation=tf.nn.relu)

            conv_2=tf.layers.conv2d(conv2,filters=256,kernel_size=[3,3],padding="same",activation=tf.nn.relu)

            pool1=tf.layers.max_pooling2d(inputs=conv_2,pool_size=[2,2],strides=[2,2],name="pool1")

            res_1=block(pool1, 256, upscale=False)

            conv3=tf.layers.conv2d(res_1,filters=256,kernel_size=[3,3],padding="same",activation=tf.nn.relu)
            pool2=tf.layers.max_pooling2d(conv3,pool_size=[2,2],strides=2,name="pool2")
            res_2=block(pool2, 256, upscale=False)
            res_3=block(res_2, 256, upscale=False)
            conv4=tf.layers.conv2d(res_3,filters=256,kernel_size=[3,3],padding="same",activation=tf.nn.relu)
            pool3=tf.layers.max_pooling2d(conv4,pool_size=[1,2],strides=[1,2],name="pool3")
            res_4=block(pool3, 256, upscale=False)
            res_5=block(res_4, 256, upscale=False)
            res_6=block(res_5, 256, upscale=False)
            res_7=block(res_6, 256, upscale=False)
            res_8=block(res_7, 256, upscale=False)
            conv_5=tf.layers.conv2d(res_8,filters=512,kernel_size=[3,3],padding="same",activation="relu")
            pool_4=tf.layers.max_pooling2d(inputs=conv_5,pool_size=[2,2],strides=[2,2],name="pool4")
            res_9=block(pool_4,512,upscale=False)
            res_10=block(res_9,512,upscale=False)
            res_11=block(res_10,512,upscale=False)
            conv_6=tf.layers.conv2d(res_11,filters=512,kernel_size=[3,3],padding="same",activation="relu")
            pool_6=tf.layers.max_pooling2d(inputs=conv_6,pool_size=[2,2],strides=[2,2],name="pool6")
            pool_7=tf.layers.max_pooling2d(inputs=pool_6,pool_size=[2,2],strides=[2,2],name="pool7")
            conv_reshaped = tf.squeeze(pool_7, axis = [1], name='reshaped')


            #b_norm6 = tf.nn.relu(tf.layers.batch_normalization(conv6, name='batch-norm3'),name='relu8')
            #conv = tf.squeeze(b_norm6, axis = [2], name='reshaped')
           # conv_reshaped = tf.squeeze(bnorm_6, axis = [1], name='reshaped')
            #list_n_hidden = [256, 256]
    num_units = 256
    num_layers = 3
    list_n_hidden = [256, 256]
    with tf.name_scope('deep_bidirectional_lstm'):
        fw_cell_list = [tf.contrib.rnn.BasicLSTMCell(nh, forget_bias=1.0) for nh in list_n_hidden]
        bw_cell_list = [tf.contrib.rnn.BasicLSTMCell(nh, forget_bias=1.0) for nh in list_n_hidden]
        lstm_net, _, _ = tf.contrib.rnn.stack_bidirectional_dynamic_rnn(fw_cell_list, bw_cell_list, conv_reshaped, sequence_length = Seq_Len,
                                                                        dtype=tf.float32)
        lstm_net = tf.nn.dropout(lstm_net, keep_prob=0.5)  # [width(time), batch, n_classes]
        shape = lstm_net.get_shape().as_list()  # [batch, width, 2*n_hidden]
        rnn_reshaped = tf.reshape(lstm_net, [-1, shape[-1]])  # [batch x width, 2*n_hidden]

        W = tf.Variable(tf.truncated_normal(shape=[512, num_classes], mean=0.0, stddev=0.02))
        b = tf.Variable(tf.constant(value=0.0, shape=[num_classes]))
        fc_out = tf.nn.bias_add(tf.matmul(rnn_reshaped, W), b)
        print(fc_out)

        lstm_out = tf.reshape(fc_out, [-1, time_step, num_classes], name='reshape_out')
        lstm_out = tf.transpose(lstm_out, [1, 0, 2], name='transpose_time_major')  # [width(time), batch, n_classes]
        print(lstm_out)
    return lstm_out

def indexing(ids):   
    CODES = {'<PAD>': 0, '<EOS>': 1, '<UNK>': 2, '<GO>': 3 }
    import copy
    vocab=set(ids)
    vocab_to_int=copy.copy(CODES)
    for vocab,integer in enumerate(vocab,len(CODES)):
        vocab_to_int[integer] = vocab
    int_to_vocab={ v_i: v for v, v_i in vocab_to_int.items()}
    return vocab_to_int,int_to_vocab
vocab_to_int,int_to_vocab=indexing(Y_train)





import tensorflow as tf

In [34]:
def Compile():
    
    Target = tf.sparse_placeholder(tf.int32, name='Target')
    

    logits =ResNetCNN()
    
    decoded, log_prob = tf.nn.ctc_beam_search_decoder(logits, Seq_Len, merge_repeated=False, beam_width=100, top_paths=2)

    ler = tf.reduce_mean(tf.edit_distance(tf.cast(decoded[0], tf.int32), Target))

    init = tf.global_variables_initializer()
    table_init = tf.tables_initializer()
    sess.run([init, table_init])
    


In [37]:
num_classes=52
from tensorflow.python.framework import ops
ops.reset_default_graph()
#input_layer = tf.placeholder(dtype=tf.float32, shape=[None, 32, params['width_thresh'], 3], name='Input')
#Outputs = tf.placeholder(tf.float32, shape = [BatchSize, 64, 2])
#logits = tf.layers.dense(dec_outputs, 2, name  = 'logits') 
#Architecture(Input, outputs)


from tensorflow.python.framework import ops
ops.reset_default_graph()
global sess

config = tf.ConfigProto()
sess = tf.Session(config = config)
graph = tf.get_default_graph()
  

In [40]:
num_classes = params['num_classes']
time_step = int(np.floor(params['width_thresh'] / 2) - 1)
Input = tf.placeholder(dtype=tf.float32, shape=[None, 32, params['width_thresh'], 3], name='Input')
Seq_Len = tf.placeholder(dtype=tf.int32, shape=[None], name='Seq_len')
from tensorflow.python.framework import ops
ops.reset_default_graph()
global sess

config = tf.ConfigProto()
sess = tf.Session(config = config)
graph = tf.get_default_graph()

Compile()

Tensor("deep_bidirectional_lstm/BiasAdd:0", shape=(?, 53), dtype=float32)
Tensor("deep_bidirectional_lstm/transpose_time_major:0", shape=(124, ?, 53), dtype=float32)


ValueError: Tensor("Seq_len:0", shape=(?,), dtype=int32) must be from the same graph as Tensor("deep_bidirectional_lstm/transpose_time_major:0", shape=(124, ?, 53), dtype=float32).